In [49]:
import pandas as pd
data = pd.read_csv('../mimic-iii-clinical-database-1.4/noteevents.csv', error_bad_lines=False);

##REMOVE THIS BEFORE SERVER
data = data[:100]

data_text = data[['TEXT']]
data_text['index'] = data_text.index
documents = data_text

/Users/Rushi/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Rushi/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [50]:
#Souce: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
#Since ubuntu server has python 3.5, Iphthon 7.10+ will not support it.
#Changed Ipython to 3.9
#pip uninstall ipython
#pip install ipython==7.9


In [124]:
#print(len(documents))
#print(documents[:5])

### Data Pre-processing

We will perform the following steps:

Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
Words that have fewer than 3 characters are removed.

All stopwords are removed.

Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.

Words are stemmed — words are reduced to their root form.


Loading gensim and nltk libraries

In [52]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Rushi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Write a function to perform lemmatize and stem preprocessing steps on the data set.

In [53]:
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

#### Select a document to preview after preprocessing.

In [125]:
#doc_sample = documents[documents['index'] == 78].values[0][0]
#print('original document: ')
#words = []
#for word in doc_sample.split(' '):
#    words.append(word)
#print(words)
#print('\n\n tokenized and lemmatized document: ')
#print(preprocess(doc_sample))

Preprocess the headline text, saving the results as ‘processed_docs’

In [56]:
processed_docs = documents['TEXT'].map(preprocess)
#processed_docs[:10]

0    [admiss, date, discharg, date, servic, addendu...
1    [admiss, date, discharg, date, date, birth, se...
2    [admiss, date, discharg, date, servic, cardiot...
3    [admiss, date, discharg, date, servic, medicin...
4    [admiss, date, discharg, date, date, birth, se...
5    [admiss, date, discharg, date, date, birth, se...
6    [admiss, date, discharg, date, servic, medicin...
7    [admiss, date, discharg, date, date, birth, se...
8    [admiss, date, discharg, date, date, birth, se...
9    [admiss, date, discharg, date, date, birth, se...
Name: TEXT, dtype: object

Bag of Words on the Data set
Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

import pickle 
 
file_pi = open('processed_docs.pkl', 'w') 
pickle.dump(processed_docs, file_pi)

import pickle

filehandler = open('processed_docs.pkl', 'r')
processed_docs_p = pickle.load(filehandler)

In [57]:
#reading a pickle file
#import pickle 
#filehandler = open(filename, 'r') 
#object = pickle.load(filehandler)

In [58]:
dictionary = gensim.corpora.Dictionary(processed_docs)

#count = 0
#for k, v in dictionary.iteritems():
#    print(k, v)
#    count += 1
#    if count > 10:
#        break

0 abdomin
1 addendum
2 admiss
3 apex
4 cavitari
5 chest
6 confirm
7 consist
8 date
9 dictat
10 discharg


## Save the dictionary

In [59]:
dictionary.save("dictionary.txtdic")

print("Saved Dictionary")
#HOW TO LOAD
#loaded_dictionary= gensim.corpora.Dictionary.load("testdict.txtdic")

### Gensim filter_extremes

Filter out tokens that appear in


less than 15 documents (absolute number) or


more than 0.5 documents (fraction of total corpus size, not absolute number).


after the above two steps, keep only the first 100000 most frequent tokens.

In [60]:
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

### Gensim doc2bow

For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [62]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
#bow_corpus[78]

[(0, 3),
 (1, 4),
 (4, 2),
 (5, 5),
 (9, 2),
 (12, 3),
 (14, 1),
 (15, 1),
 (16, 1),
 (18, 4),
 (19, 4),
 (20, 3),
 (22, 2),
 (23, 1),
 (24, 3),
 (25, 1),
 (29, 1),
 (33, 1),
 (34, 2),
 (37, 1),
 (39, 1),
 (40, 1),
 (42, 1),
 (43, 1),
 (45, 3),
 (48, 1),
 (54, 2),
 (56, 1),
 (57, 1),
 (58, 5),
 (62, 1),
 (63, 3),
 (64, 3),
 (65, 2),
 (66, 2),
 (68, 2),
 (69, 1),
 (73, 2),
 (74, 1),
 (76, 2),
 (77, 4),
 (78, 1),
 (79, 2),
 (80, 2),
 (83, 1),
 (84, 1),
 (86, 1),
 (89, 1),
 (91, 1),
 (93, 2),
 (94, 1),
 (95, 1),
 (96, 1),
 (101, 1),
 (102, 3),
 (103, 3),
 (104, 2),
 (105, 2),
 (106, 1),
 (109, 4),
 (114, 3),
 (119, 1),
 (120, 1),
 (121, 1),
 (126, 4),
 (127, 2),
 (128, 1),
 (131, 3),
 (135, 1),
 (136, 1),
 (137, 1),
 (138, 2),
 (139, 1),
 (141, 5),
 (143, 2),
 (144, 1),
 (145, 1),
 (146, 2),
 (152, 4),
 (154, 1),
 (156, 3),
 (157, 1),
 (158, 3),
 (159, 1),
 (160, 3),
 (161, 1),
 (162, 3),
 (163, 3),
 (167, 2),
 (170, 2),
 (171, 1),
 (172, 1),
 (176, 5),
 (179, 1),
 (183, 1),
 (184, 1),
 (

file_pi1 = open('bow_corpus.pkl', 'w') 
pickle.dump(bow_corpus, file_pi1)

### Preview Bag Of Words for our sample preprocessed document.

In [126]:
#bow_doc_78 = bow_corpus[78]
#for i in range(len(bow_doc_78)):
#    print("Word {} (\"{}\") appears {} time.".format(bow_doc_78[i][0], 
#                                               dictionary[bow_doc_78[i][0]], 
#bow_doc_78[i][1]))

### TF-IDF
Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [127]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint

#for doc in corpus_tfidf:
#    pprint(doc)
#    break

### Running LDA using Bag of Words

Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [70]:
#KEEP THIS FOR SERVER

#lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=50, id2word=dictionary, passes=2, workers=2)

print('Training LDA BOW')
lda_model_bow = gensim.models.LdaMulticore(bow_corpus, num_topics=50, id2word=dictionary, passes=2, workers=2)



Process ForkPoolWorker-35:
Process ForkPoolWorker-34:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/Users/Rushi/anaconda/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 334, in worker_e_step
    chunk_no, chunk, worker_lda

KeyboardInterrupt: 

  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


For each topic, we will explore the words occuring in that topic and its relative weight.

In [ ]:
for idx, topic in lda_model_bow.print_topics(-1):
    print('LDA Topic: {} \nWords: {}'.format(idx, topic))

## Save the LDA bow model

In [83]:
lda_model_bow.save("lda_bow.model")


#HOW TO LOAD
#saved_model_bow = gensim.models.LdaMulticore.load("lda_bow.model")

Can you distinguish different topics using the words in each topic and their corresponding weights?

### Running LDA using TF-IDF

In [68]:
#CHANGE NUM TOPICS TO 50

print('Training LDA TFIDF')
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=50, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('TFIDF Topic: {} Word: {}'.format(idx, topic))

Process ForkPoolWorker-21:
Process ForkPoolWorker-20:
Process ForkPoolWorker-19:
Process ForkPoolWorker-18:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/Rushi/anaconda/lib/

KeyboardInterrupt: 

  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/Users/Rushi/anaconda/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt


## Save the LDA tfidf model

In [84]:
lda_model_tfidf.save("lda_tfidf.model")


#HOW TO LOAD
#saved_model_tfidf = gensim.models.LdaMulticore.load("lda_tfidf.model")

Again, can you distinguish different topics using the words in each topic and their corresponding weights?


### Performance evaluation by classifying sample document using LDA Bag of Words model


We will check where our test document would be classified.

In [ ]:
#for index, score in sorted(lda_model_bow[bow_corpus[78]], key=lambda tup: -1*tup[1]):
#    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_bow.print_topic(index, 10)))

Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

### Performance evaluation by classifying sample document using LDA TF-IDF model.

In [ ]:
#for index, score in sorted(lda_model_tfidf[bow_corpus[78]], key=lambda tup: -1*tup[1]):
#    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

## How to get the top two scores in a model

Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.


### Testing model on unseen document

In [23]:
#unseen_document = 'How a Pentagon deal became an identity crisis for Google'
#bow_vector = dictionary.doc2bow(preprocess(unseen_document))
#for index, score in sorted(lda_model_bow[bow_vector], key=lambda tup: -1*tup[1]):
#    print("Score: {}\t Topic: {}".format(score, lda_model_bow.print_topic(index, 5)))

Score: 0.20000000298023224	 Topic: 0.086*"leav" + 0.041*"right" + 0.039*"previous" + 0.033*"clip" + 0.029*"chang"
Score: 0.20000000298023224	 Topic: 0.039*"normal" + 0.039*"right" + 0.033*"leav" + 0.032*"assess" + 0.030*"blood"
Score: 0.20000000298023224	 Topic: 0.045*"continu" + 0.042*"remain" + 0.042*"care" + 0.034*"activ" + 0.033*"know"
Score: 0.20000000298023224	 Topic: 0.037*"plan" + 0.036*"note" + 0.034*"monitor" + 0.033*"assess" + 0.033*"pain"
Score: 0.20000000298023224	 Topic: 0.037*"clip" + 0.035*"reason" + 0.034*"imag" + 0.032*"right" + 0.032*"leav"


### Things to Save NEXT STEPS

### Remove all print commands - Done

### Save output to a csv file - Done

### Sample run a .py file - Doing

In [77]:
savedf = pd.DataFrame()

savedf['INDEX'], savedf['ROW_ID'], savedf['SUBJECT_ID'], savedf['HADM_ID'], savedf['TEXT'] = data.index, data['ROW_ID'], data['SUBJECT_ID'], data['HADM_ID'], data['TEXT']



In [80]:
savedf['BOW_TOPIC'], savedf['BOW_SCORE'], savedf['TFIDF_TOPIC'], savedf['TFIDF_SCORE'] = np.nan, np.nan, np.nan, np.nan



In [114]:
textnum=0
for textnum in range(0, len(bow_corpus)):
    savedf['BOW_TOPIC'].loc[(savedf['INDEX']==textnum)] = sorted(lda_model_bow[bow_corpus[textnum]], key=lambda tup: -1*tup[1])[0][0]
    savedf['BOW_SCORE'].loc[(savedf['INDEX']==textnum)] = sorted(lda_model_bow[bow_corpus[textnum]], key=lambda tup: -1*tup[1])[0][1]
    savedf['TFIDF_TOPIC'].loc[(savedf['INDEX']==textnum)] = sorted(lda_model_tfidf[bow_corpus[textnum]], key=lambda tup: -1*tup[1])[0][0]
    savedf['TFIDF_SCORE'].loc[(savedf['INDEX']==textnum)] = sorted(lda_model_tfidf[bow_corpus[textnum]], key=lambda tup: -1*tup[1])[0][1]
    
    

KeyError: 'LDA_TOPIC'

In [117]:
savedf.to_csv("lda_topic_noteevents.csv")

print("Done Saving in a CSV file!")



## Things to change

dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)


data = data[:100]

use "cat nohup.out"